<a href="https://colab.research.google.com/github/pcbzmani/Learning-repository-for-scala-and-pyspark/blob/main/Solutions/Python/multiplejoinswithfilter.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install pyspark

     |████████████████████████████████| 281.4 MB 35 kB/s 
     |████████████████████████████████| 198 kB 46.7 MB/s 
  Created wheel for pyspark: filename=pyspark-3.2.1-py2.py3-none-any.whl size=281853642 sha256=d91cdcfcad680b42ea9f3384015783971b2931efd267047a63e98241034eb2e7
  Stored in directory: /root/.cache/pip/wheels/9f/f5/07/7cd8017084dce4e93e84e92efd1e1d5334db05f2e83bcef74f
Successfully built pyspark


In [2]:
from pyspark.sql import SparkSession
spark = SparkSession.builder\
        .master("local")\
        .appName("datediff")\
        .config('spark.ui.port', '4040')\
        .getOrCreate()

In [20]:
emp_data = [(1,'Mani',1,3),(2,'Pat',2,2),(3,'Raj',3,1),(4,'Vinoth',4,4)]
cols_emp = ['id','name','dept_id','grade_id']
dept_data = [(1,'IT'),(2,'Data Analytics'),(3,'Infra'),(4,'Cyber')]
cols_dept = ['id','name']
grade_data = [(1,'Trainee'),(2,'Associate'),(3,'Senior Associate'),(4,'Consultant')]
cols_grade = ['id','name']

In [21]:
rdd1 = spark.sparkContext.parallelize(emp_data)
emp_df = rdd1.toDF(cols_emp)

rdd2 = spark.sparkContext.parallelize(dept_data)
dept_df = rdd2.toDF(cols_dept)

rdd3 = spark.sparkContext.parallelize(grade_data)
grade_df = rdd3.toDF(cols_grade)

In [22]:
emp_df.show()

+---+------+-------+--------+
| id|  name|dept_id|grade_id|
+---+------+-------+--------+
|  1|  Mani|      1|       3|
|  2|   Pat|      2|       2|
|  3|   Raj|      3|       1|
|  4|Vinoth|      4|       4|
+---+------+-------+--------+



In [23]:
dept_df = dept_df.withColumnRenamed('id','dept_id')\
                 .withColumnRenamed('name','dept_name')

In [24]:
grade_df = grade_df.withColumnRenamed('id','grade_id')\
                   .withColumnRenamed('name','grade_name')

In [25]:
dept_join_cond = [emp_df.dept_id == dept_df.dept_id]
grade_join_cond = [emp_df.grade_id == grade_df.grade_id]
join_type = 'left_outer'

In [32]:
res_df = emp_df.join(dept_df,dept_join_cond,join_type)\
               .join(grade_df,grade_join_cond,join_type)\
               .filter('dept_name =="Infra" or dept_name == "IT" or grade_name == "Associate"')\
               .select('id','name','dept_name','grade_name')

In [33]:
res_df.show()

+---+----+--------------+----------------+
| id|name|     dept_name|      grade_name|
+---+----+--------------+----------------+
|  3| Raj|         Infra|         Trainee|
|  2| Pat|Data Analytics|       Associate|
|  1|Mani|            IT|Senior Associate|
+---+----+--------------+----------------+



In [34]:
spark.stop()